# 走行２　予選　駐車あり

１カメラを使用して電光掲示板の指示よりモデルを切り替えて走行します。３周したらパーキング

cam0 走行用（ステアリング、アクセル）

cam0 方向指示用および周回数カウンタ用（環境判断）

ブレーキ、バックを有効化しており、従来とSpeedアノテーション（スロットル）に位置変化とパラメータ変化に注意すること

タミヤ製のESCでバックするには、一旦ニュートラル信号状態にしてバック信号にする必要がある。PCA9685は、水晶発振をつかわない場合は、ニュートラル信号は正確な値に調整しなくはいけない。

対応デバイス: Jetson Orin Nano

Revision0.0.1

Release 2024/10/20

## Jetsonの認識

In [1]:
import Jetson.GPIO as GPIO

BOARD_NAME = GPIO.gpio_pin_data.get_data()[0]

mode_descriptions = {
    "JETSON_NX": ["15W_2CORE", "15W_4CORE", "15W_6CORE", "10W_2CORE", "10W_4CORE"],
    "JETSON_XAVIER": ["MAXN", "MODE_10W", "MODE_15W", "MODE_30W"],
    "JETSON_NANO": ["MAXN", "5W"],
    "JETSON_ORIN": ["MAXN", "MODE_15W", "MODE_30W", "MODE_40W"],
    "JETSON_ORIN_NANO": ["MODE_15W", "MODE_7W"]
}

product_names = {
    "JETSON_NX": "Jetson Xavier NX",
    "JETSON_XAVIER": "Jetson AGX Xavier",
    "JETSON_NANO": "Jetson Nano",
    "JETSON_ORIN": "Jetson AGX Orin",
    "JETSON_ORIN_NANO": "Jetson Orin Nano"
}

# ボードごとのI2Cバス番号と初期Powerモードを定義する
board_settings = {
    "JETSON_NX": (8, 3),
    "JETSON_XAVIER": (8, 2),
    "JETSON_NANO": (1, 0),
    "JETSON_ORIN": (7, 0),
    "JETSON_ORIN_NANO": (7, 0)
}

i2c_busnum, power_mode = board_settings.get(BOARD_NAME, (None, None))
mode_description = mode_descriptions.get(BOARD_NAME, [])
product_name = product_names.get(BOARD_NAME, "未知のボード")

if power_mode is not None and power_mode < len(mode_description):
    mode_str = mode_description[power_mode]
    print("------------------------------------------------------------")
    print(f"{product_name}を認識: I2Cバス番号: {i2c_busnum}, Powerモード: {mode_str}({power_mode})に設定します。")
    print("------------------------------------------------------------")
else:
    print("未知のボードまたは不正なモードです。")

------------------------------------------------------------
Jetson Orin Nanoを認識: I2Cバス番号: 7, Powerモード: MODE_15W(0)に設定します。
------------------------------------------------------------


In [2]:
if (product_name == "Jetson Orin Nano") or (product_name == "Jetson AGX Orin"):
    print("Docker起動のため電力モードは変更できません。")
else:
    !echo "jetson" | sudo -S nvpmodel -m $power_mode

Docker起動のため電力モードは変更できません。


In [3]:
!echo "jetson" | sudo -S nvpmodel -q

NV Power Mode: 15W
0


In [4]:
if (product_name == "Jetson Orin Nano") or (product_name == "Jetson AGX Orin"):
    print("Docker起動のためjetson_clocksは起動できません。")
else:
    !echo "jetson" | sudo -S jetson_clocks

Docker起動のためjetson_clocksは起動できません。


## ログの表示用,ログファイル Widget

In [5]:
import ipywidgets
from ipywidgets import Button, Layout, Textarea, HBox, VBox, Label
import os
import glob
from IPython.display import clear_output
import traceback

l = Layout(flex='0 1 auto', height='100px', min_height='100px', width='auto')
process_widget = ipywidgets.Textarea(description='ログ', value='', layout=l)

#ログのプロセスナンバー
process_no = 0
#デバッグモード有効無効（ユーザーが必要に応じて以下をTrueかFalseに変更する）
DEBUG = False

def write_log(msg):
    global process_widget, process_no
    process_no = process_no + 1
    log_message = f"{process_no}: {msg}\n"
    process_widget.value = log_message + process_widget.value
    
    # ログファイルに書き込む
    if DEBUG:
        with open("/home/jetson/data/notebooks/logfile.log", "a") as log_file:
            log_file.write(log_message)
        
    # UIのクリアと更新
    clear_output(wait=True)

## PWMの値の読み込み

In [6]:
import Fabo_PCA9685
import time
import pkg_resources
import smbus
import time
import json

#PCA9685からPWM信号を出力し、PCA9685は解像度12bit(0~4095)周波数60Hzで初期、論理値375（1500u秒）で0ch,1chを出力する。
#Jetson Orin Nanoはバス番号は7、PCA9685のI2Cデバイスアドレスは0x40
SMBUS='smbus'
BUSNUM=i2c_busnum
SERVO_HZ=60
INITIAL_VALUE=375
bus = smbus.SMBus(BUSNUM)
PCA9685 = Fabo_PCA9685.PCA9685(bus,INITIAL_VALUE,address=0x40)
PCA9685.set_hz(SERVO_HZ)

#0chは、ステアリング、1chは、スロットル　ESCの設定やメーカーによって反対なので注意すること
STEERING_CH = 0
THROTTLE_CH = 1
direction = 0
REVERSE = 0
NORMAL = 1

pwm_front = 0
pwm_back = 0

#ステアリング、スロットルのPWM波長の設定値ファイルを読み込む。

with open('pwm_params.json') as f:
    json_str = json.load(f)
    
    pwm_stop = json_str["pwm_speed"]["stop"]
    pwm_front = json_str["pwm_speed"]["front"]
    pwm_back = json_str["pwm_speed"]["back"]
    pwm_left = json_str["pwm_steering"]["left"]
    pwm_center = json_str["pwm_steering"]["center"]
    pwm_right = json_str["pwm_steering"]["right"]

#信号ロジック切り替え　ESCメーカー、ESC設定による対応
if pwm_front >= pwm_back:
    direction = REVERSE
else:
    direction = NORMAL

#RCカー電源投入時の初期信号    
PCA9685.set_channel_value(STEERING_CH, pwm_center)
PCA9685.set_channel_value(THROTTLE_CH, pwm_stop)

## カメラの読込

この部分でエラーが発生する場合は、Jetsonの再起動をお願いします。<br>
２２４×２２４ドットで画像を取り込む。<br>
カメラが認識できない場合は、ケーブルの接続確認をしてください。


In [7]:
#各カメラのFPS設定
CAM0_FPS=60

fps_type = CAM0_FPS

In [8]:
from jetcam.csi_camera import CSICamera

def open_camera():
    global cam0
    try:
        cam0 = CSICamera(capture_device=0,width=224, height=224, capture_fps=CAM0_FPS)
    except Exception as e:
        # スタックトレースを含むエラーメッセージを取得
        error_message = f"Error open_camera:{e}\n{''.join(traceback.format_exception(None, e, e.__traceback__))}"
        write_log(error_message)

###　メイン処理（画像推論など）

ログ出力できないので、カメラは別スレッドで行う。

In [9]:
import torch
import threading
from utils import preprocess
import subprocess
import cv2
import time
from torch2trt import TRTModule
import subprocess
import datetime
import torch.nn.functional as F

IMG_WIDTH=224

record = False
running = False
load = False
speed_ai_flag = False
running_cam0 = False

lap = 0
modelChange = False
select_model_name = "left" #初期モデルはleft
total_process_detect_time = 0

def map_rc(x, in_min, in_max, out_min, out_max):
    return (x - in_min) * (out_max - out_min) // (in_max - in_min) + out_min
#ハンドル数値変換　ドットからPWM信号(12bit)値へ変換
def handle(x):
    global pwm_right,pwm_left,STEERING_CH,PCA9685
    x = map_rc(x, 224, 0, pwm_right, pwm_left)
    PCA9685.set_channel_value(STEERING_CH, x)
#スロットル数値変換　ドットからPWM信号(12bit)値へ変換   
def throttle(speed):
    global pwm_front,pwm_back,THROTTLE_CH,PCA9685, low_gain
    speed = map_rc(speed, 224, 0, pwm_front, pwm_back + speed_low_gain_slider.value)
    PCA9685.set_channel_value(THROTTLE_CH, speed)

##カメラ０のタスク走行カメラ（１つのカメラで走行推論、環境判断をする）  
def live_cam0():
    global save_dir0,total_process_detect_time,modelChange,select_model_name,lap,cam0,speed_ai_flag,running_cam0,camera,IMG_WIDTH,record,model_a_trt,model_b_trt,model_c_trt,preprocess,pwm_stop,save_dir,num,count,fps_type,FPS_30
    
    count = 1
    num = 1
    frame_count = 0

    start_time = time.time()
    process_drive_time = 0
    process_detect_time = 0
    total_process_drive_time = 0
    
    #画像ファイル名カウンタ
    count_cam0 = 1
    
    last_lap_time = 0
    process_detect_time = 0
    last_detect = 0
    detect_count = 0
    
    #起動時のデフォルト推論モデル
    model = model_a_trt
    
    #カメラ0のタスク（環境推論してからドライビング推論をくりかえす。）
    while running_cam0:
        try:
            #カメライメージと取り込む
            image = cam0.read()
            
            #カメラの１８０度回転
            #image = cv2.rotate(image, cv2.ROTATE_180)
            
            #環境推論実施
            image = preprocess(image2).half()
            output = model_c_trt(image2).detach()
            output = F.softmax(output, dim=1).cpu().numpy().flatten()
            category_index = output.argmax()
            
            #現在の時間を取得
            current_time = time.time()
            
            #分岐路電光掲示板指示判定 カテゴリー０であれば左のモデルを読む、カテゴリー１であれば右へ行くモデルを読む。カテゴリー３はETC（モデルはそのまま）
            #環境判定が同じモデルならば入れ替えない。
            if category_index == 0 and select_model_name == "right": 
                modelChange = True
                select_model_name = "left"
            elif category_index == 1 and select_model_name == "left":
                modelChange = True
                select_model_name = "right"
            elif category_index == 3:
                #モデルの入れ替えはしない。
                modelChange = False
                #write_log("モデルetc")
            
            # 環境判断によるLapカウンタ　カテゴリーナンバー2であればスタートラインであると判断する。
            if current_time - last_lap_time > 5:  # 5秒経過後に判定を再開
                if category_index == 2 :
                    detect_count += 1
                    if detect_count >= 3:
                        lap += 1
                        detect_count = 0
                        last_lap_time = current_time  # 計測時間をリセット
                    else:
                        detect_count = 0

                #last_detect = category_index
                
            #環境情報の認識時間
            total_process_detect_time = time.time() - process_detect_time 
            
            # メインカメラ画像を保存
            if record == True:
                name = "0_0_{:0=5}.jpg".format(count_cam0)
                image_path0 = os.path.join(save_dir0, name)
                cv2.imwrite(image_path0, image)

            # カウンターを増加
            count_cam0 += 1
            
            #画像読み込み
            process_drive_time = time.time()
            
            # 実際のモデル入れ替え処理　３周以上になったらドライビングモデルから駐車モデルに入れ替える
            if lap >= 3:
                #駐車モデル入れ替えをする
                model = model_p_trt
                select_model_name = "parking"
            else:
                #分岐路モデルチェンジする処理
                if modelChange == True:
                    if select_model_name == "left":
                        #モデル内回り入れ替えをする
                        model = model_a_trt
                        modelChange = False
                    elif select_model_name == "right":
                        #モデル外回り入れ替えをする
                        model = model_b_trt
                        modelChange = False
                    else:
                        #etc
                        write_log(f"modelChange:{modelChange}")
                        model = model_a_trt
                #else:
                     #write_log(f"Change:{modelChange}")

            
            #ドライビング推論を実施 yは推論使用しない。
            output = model(image).detach().cpu().numpy().flatten()
            x = float(output[0]) * steering_gain_slider.value
            y = float(output[1]) #y値は使用しない。
            x = int(IMG_WIDTH * (x / 2.0 + 0.5))
            speed = int(IMG_WIDTH * (y / 2.0 + 0.5)) * speed_gain_slider.value
            
            #操舵推論結果をPCA9685によるPWM出力
            handle(x)
            
            #スロットル推論有効の場合
            if speed_ai_flag == False:
                speed = speed_raw_slider.value
            else:
                if speed > 224:
                    speed = 224
                    
            #スロットル推論結果をPCA9685によるPWM出力
            throttle(speed)
            
            total_process_drive_time += time.time() - process_drive_time
                 
        except Exception as e:
            write_log(f"Error-cam0:{e}")
                       
            #ドライビング推論時間記録
            process_detect_time = time.time()
          
            count += 1
            frame_count += 1

            # 走行時の処理時間計測
            if time.time() - start_time > 3.0:
                fps = frame_count / 3.0
                speed_type = ""
                if speed_ai_flag == False:
                    speed_type = f"(固定)"
                else:
                    speed_type = f"(推論)"

                write_log(f"Lap: {lap}, Model: {select_model_name}, Speed: {speed:.1f} {speed_type}, Steering Gain: {steering_gain_slider.value}, FPS: {fps:.1f}, 走行推論: {total_process_drive_time/(fps*3)*1000:.1f}ms, 環境推論: {total_process_detect_time:.1f}ms")
                frame_count = 0
                start_time = time.time() 
                total_process_detect_time = 0
                total_process_drive_time = 0
        except Exception as e:
            write_log(f"Error_Cam0:{e}")
            
    if record == True:
        if fps_type == 30:
            write_log(f"FPS30走行画像を{count}枚の走行データを保存しました。")
        else:
            write_log(f"FPS60走行画像を{num}枚の走行データを保存しました。")

### カメラ起動処理
def start_cameras():
    global cam0, running_cam0, execute_thread_cam0
    
    open_camera()
    
    # Cam0を起動　別スレッドで起動。
    running_cam0 = True
    execute_thread_cam0 = threading.Thread(target=live_cam0)
    execute_thread_cam0.start()
    write_log("Start cam0")
    
#指定されたディレクトリに保存
def setup_save_directory(base_path):
    if not os.path.exists(base_path):
        subprocess.call(['mkdir', '-p', base_path])
    write_log(f"{base_path}にデータを保存します。")
     
#runボタンが押された場合の処理       
def run(change):
    global running,execute_thread,name_widget,save_dir,start_time,load_a,load_b,load_c,save_dir0
    
    #各モデルセット完了チェック
    if load_a == False or load_b == False or load_c == False or load_p == False:
        write_log("モデルが読み込まれていません")
        return
    #AIが起動していない場合
    if not running:
        write_log("AIが起動しました。")

        #録画が有効な場合
        if record:
            #ファイル名が記入済み
            if name_widget.value != "":
                #デフォルトパス
                base_path = "camera/" + name_widget.value
                # Cam0の保存先を設定
                save_dir0 = f"{base_path}_cam0/xy/"
                #ディレクトリを設定
                setup_save_directory(save_dir0)
            else:
                write_log("【Error】 映像の保存先を入力してください。")
                return

        #runボタンが押されたならカメラを起動する。
        write_log("カメラを起動中...")
        start_cameras()
        start_time = time.time()

#stopボタンが押された場合
def stop(change):
    #global running_cam0,execute_thread,end_time,start_time,count,pwm_stop
    global running,running_cam0,execute_thread_cam0,end_time,start_time,pwm_stop
    
    if running_cam0 == True:
        PCA9685.set_channel_value(THROTTLE_CH, pwm_stop)
        try:
            end_time = time.time() - start_time
            fps = count/int(end_time)
            process_time = int((end_time/count)*1000)
        except:
            fps = -1
            process_time = -1
        
        write_log("AIを停止しました。")
        write_log("処理結果:FPS: " + str(round(fps,2)) + ",処理回数: " + str(count) + ",　処理時間(1回平均値): " + str(process_time) + " ms")
        running = False
        running_cam0 = False
        
        #スレッド削除
        try:    
            execute_thread_cam0.join()
            write_log("カメラスレッド削除。")
        except Exception as e:
            write_log(f"Thread joinでエラー(すでにthreadが存在しない:{e}")
            
        try:
            stop_camera(None)
        except Exception as e:
            write_log(f"カメラの停止処理でエラー:{e}")
            
    else:
        PCA9685.set_channel_value(THROTTLE_CH, pwm_stop)
        write_log("現在AIは動いていません。")

### UI(wigeht)の定義

In [10]:
model_a_widget = ipywidgets.Dropdown(options=[],description='モデル')
model_a_time_widget = ipywidgets.Label(description='作成日時')
load_a_button = ipywidgets.Button(description='走行モデルを読込み')

model_b_widget = ipywidgets.Dropdown(options=[],description='モデル')
model_b_time_widget = ipywidgets.Label(description='作成日時')
load_b_button = ipywidgets.Button(description='走行モデルを読込み')

model_p_widget = ipywidgets.Dropdown(options=[],description='モデル')
model_p_time_widget = ipywidgets.Label(description='作成日時')
load_p_button = ipywidgets.Button(description='走行モデルを読込み')

model_c_widget = ipywidgets.Dropdown(options=[],description='モデル')
model_c_time_widget = ipywidgets.Label(description='作成日時')
load_c_button = ipywidgets.Button(description='環境モデルを読込み')

run_button = ipywidgets.Button(description='走行開始')
stop_button = ipywidgets.Button(description='停止')

pwm_left_widget = ipywidgets.IntText(value=pwm_left,description='PWM 左')
pwm_center_widget = ipywidgets.IntText(value=pwm_center,description='PWM 中央')
pwm_right_widget = ipywidgets.IntText(value=pwm_right,description='PWM 右')

check_left_button = ipywidgets.Button(description='チェック左')
check_center_button = ipywidgets.Button(description='チェック中央')
check_right_button = ipywidgets.Button(description='チェック右')

pwm_stop_widget = ipywidgets.IntText(value=pwm_stop,description='PWM 停止')

name_widget = ipywidgets.Text(description='映像の保存名')
record_box = ipywidgets.Checkbox(False, description='録画')
speed_low_gain_slider = ipywidgets.FloatSlider(description='Speed Low gain', min=0, max=20, step=1, value=20, orientation='horizontal')
speed_gain_slider = ipywidgets.FloatSlider(description='Speed gain', min=0.1, max=3.0, step=0.1, value=1.0, orientation='horizontal')
speed_raw_slider = ipywidgets.IntSlider(description='Speed raw', min=1, max=224, step=1, value=80, orientation='horizontal')
steering_gain_slider = ipywidgets.FloatSlider(description='Steering gain', min=0.1, max=2.0, step=0.1, value=1.0, orientation='horizontal')
speed_dropbox = ipywidgets.Dropdown(options=["固定値","推論値"], description='Speed')
release_button = ipywidgets.Button(description='Camera開放')

### モデルロード、モデル選択処理　ウィジェットのイベント時に呼ばれる関数定義

In [11]:
#ウィジェットの内容の選択が変化したときに呼ばれる
def load_a_model(change):
    global model_a_trt,load_a
    try:
        write_log(model_a_widget.value + "の読込を実行します(初回は時間がかかります)。")
        model_a_trt = TRTModule()
        model_a_trt.load_state_dict(torch.load(model_a_widget.value))
        write_log(model_a_widget.value + "の読込に成功しました。")
        load_a = True
        get_jetson_nano_memory_usage()
    except Exception as e:
        write_log(f"【Error-a】 {e} : {model_a_widget.value} の読込に失敗しました。")

load_a_button.on_click(load_a_model)   

def load_b_model(change):
    global model_b_trt,load_b
    try:
        write_log(model_b_widget.value + "の読込を実行します(初回は時間がかかります)。")
        model_b_trt = TRTModule()
        model_b_trt.load_state_dict(torch.load(model_b_widget.value))
        write_log(model_b_widget.value + "の読込に成功しました。")
        load_b = True
        get_jetson_nano_memory_usage()
    except Exception as e:
        write_log(f"【Error-b】 {e} : {model_b_widget.value} の読込に失敗しました。")

load_b_button.on_click(load_b_model)

def load_p_model(change):
    global model_p_trt,load_p
    try:
        write_log(model_p_widget.value + "の読込を実行します(初回は時間がかかります)。")
        model_p_trt = TRTModule()
        model_p_trt.load_state_dict(torch.load(model_p_widget.value))
        write_log(model_p_widget.value + "の読込に成功しました。")
        load_p = True
        get_jetson_nano_memory_usage()
    except Exception as e:
        write_log(f"【Error-p】 {e} : {model_p_widget.value} の読込に失敗しました。")

load_p_button.on_click(load_p_model)

def load_c_model(change):
    global model_c_trt,load_c
    try:
        write_log(model_c_widget.value + "の読込を実行します(初回は時間がかかります)。")
        model_c_trt = TRTModule()
        model_c_trt.load_state_dict(torch.load(model_c_widget.value))
        write_log(model_c_widget.value + "の読込に成功しました。")
        load_c = True
        get_jetson_nano_memory_usage()
    except Exception as e:
        write_log(f"【Error-c】 {e} : {model_c_widget.value} の読込に失敗しました。")

load_c_button.on_click(load_c_model) 

def model_list(type):
    try:
        files = glob.glob(type + '/*.pth', recursive=True)
        #ts = os.path.getctime(files[0])
        #d = datetime.datetime.fromtimestamp(ts)
        #s = d.strftime('%Y-%m-%d %H:%M:%S')
        if type == "model_trt":
            model_a_widget.options = files
            #model_a_time_widget.value = s
            model_b_widget.options = files
            #model_b_time_widget.value = s
            model_p_widget.options = files
        elif type == "model_class_trt":
            model_c_widget.options = files
            #model_c_time_widget.value = s
    except Exception as e:
        model_a_widget.options = []
        model_b_widget.options = []
        model_c_widget.options = []
        #write_log(f"Error:{e}")
model_list("model_trt")
model_list("model_class_trt")

def change_a_file(change):
    file = model_a_widget.value
    ts = os.path.getctime(file)
    d = datetime.datetime.fromtimestamp(ts)
    s = d.strftime('%Y-%m-%d %H:%M:%S')
    model_a_time_widget.value = s
model_a_widget.observe(change_a_file)

def change_b_file(change):
    file = model_b_widget.value
    ts = os.path.getctime(file)
    d = datetime.datetime.fromtimestamp(ts)
    s = d.strftime('%Y-%m-%d %H:%M:%S')
    model_b_time_widget.value = s
model_b_widget.observe(change_b_file)

def change_p_file(change):
    file = model_p_widget.value
    ts = os.path.getctime(file)
    d = datetime.datetime.fromtimestamp(ts)
    s = d.strftime('%Y-%m-%d %H:%M:%S')
    model_p_time_widget.value = s
model_p_widget.observe(change_p_file)

def change_c_file(change):
    file = model_c_widget.value
    ts = os.path.getctime(file)
    d = datetime.datetime.fromtimestamp(ts)
    s = d.strftime('%Y-%m-%d %H:%M:%S')
    model_c_time_widget.value = s
model_c_widget.observe(change_c_file)

#録画チェックが変化したときに呼ばれる
def on_video(change):
    global record
    record^=True

def on_fixed_value_change(change):
    global speed_ai_flag, speed_gain_slider, speed_raw_slider
    if change['new'] == "固定値":
        speed_gain_slider.layout.visibility = 'hidden'
        speed_raw_slider.layout.visibility = 'visible'
        speed_gain_slider.disabled = True
        speed_raw_slider.disabled = False
        speed_ai_flag = False
    elif change['new'] == "推論値":
        speed_gain_slider.layout.visibility = 'visible'
        speed_raw_slider.layout.visibility = 'hidden'
        speed_gain_slider.disabled = False
        speed_raw_slider.disabled = True
        speed_ai_flag = True

#表示に関するプロパティ
speed_gain_slider.layout.visibility = 'hidden'
speed_raw_slider.layout.visibility = 'visible'
speed_gain_slider.disabled = True
speed_raw_slider.disabled = False
speed_dropbox.observe(on_fixed_value_change, names='value')

#イベント定義
run_button.on_click(run)
stop_button.on_click(stop)
record_box.observe(on_video)

### SDカードの残り容量を調べる

※残り残量が少なくなった場合は不要なファイルを削除する

In [12]:
import subprocess
import re

used_memory_widget = ipywidgets.IntText(description='Useメモリ', value=1)
total_memory_widget = ipywidgets.IntText(description='全メモリ', value=1)
memory_button = ipywidgets.Button(description='使用メモリ量の取得')

def get_jetson_nano_memory_usage(event=None):
    command = 'tegrastats'
    try:
        process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, universal_newlines=True)
        
        mem_usage_pattern = re.compile(r'RAM (\d+)/(\d+)MB')
        
        max_lines_to_read = 10
        for _ in range(max_lines_to_read):
            line = process.stdout.readline()
            if not line:
                break 
            matches = mem_usage_pattern.search(line)
            if matches:
                used_memory_widget.value = int(matches.group(1))
                total_memory_widget.value = int(matches.group(2))
                process.kill()
                return
        
        process.kill()  
        return

    except subprocess.CalledProcessError as e:
        return

get_jetson_nano_memory_usage()
memory_button.on_click(get_jetson_nano_memory_usage)

### カメラ開放

In [13]:
import time

def stop_camera(c):
    global cam0
    cam0.running = False
    time.sleep(0.1)
    cam0.cap.release()
    write_log("カメラ0を開放しました。")

release_button.on_click(stop_camera)

走行までの流れは以下の通りです。

1. <b>走行モデルを指定してLoadする</b><br>
2. <b>環境モデルを指定してLoadする</b><br>
3. <b>Speedに固定値, Speedに推論値のいずれかを選択する</b><br>
固定値を選んだ場合は固定の速度で走り続けます。推論値を選んだ推論結果を速度に反映します。速度用のアノテーションは13_annotation.ipynbで追加可能です。<br>
4. <b>[オプション] 走行動画を録画する場合は、録画にチェックマークをいれて、保存ファイル名を指定する</b><br>
./runフォルダに保存<br>
5. <b>走行開始ボタンを押して、プロポの裏側のボタンを押して AIモードで自動走行開始する</b><br>
6. <b>終了時は、停止ボタンを押す</b><br>
録画のチェックマークがついている場合は、停止で録画も終了<br>
Speed Inferenceのチェックマークがついている場合は、スロットル量の推論も有効になる<br>
<br>
カメラが60fpsで動いている場合は16ms以内、カメラが30fpsで動いている場合は、33ms以内での処理完了が正常な挙動となります。

### ウィジェットの配置と表示

In [14]:
separator = ipywidgets.HTML('<hr style="border-color:gray;margin:10px 0"/>')
title1 = ipywidgets.HTML('<b>【1.使用する走行モデルA(LEFT内回り用)】</b> TensoRTに変換済みのモデルをLoadします。')
title2 = ipywidgets.HTML('<b>【2.使用する走行モデルB(RIGHT外回り用)】</b> TensoRTに変換済みのモデルをLoadします。')
title3 = ipywidgets.HTML('<b>【3.使用する走行モデルC(駐車モデル)】</b> TensoRTに変換済みのモデルをLoadします。')
title4 = ipywidgets.HTML('<b>【4.使用する電光掲示板分岐判断環境モデルC】</b> TensoRTに変換済みのモデルをLoadします。')
title5 = ipywidgets.HTML('<b>【5.Steeringゲイン】</b> Steeringのゲイン調整します。周りが悪い時は値を1.0以上にします。')
title6 = ipywidgets.HTML('<b>【6.速度】</b> 速度は固定値か、推論から反映かが選べます。')
title7 = ipywidgets.HTML('<b>【7.録画】</b> 走行中の映像を録画したい場合はチェックマークを選択し、保存データセット名を指定してください。')
title8 = ipywidgets.HTML('<b>【8.走行の開始】</b> 走行開始を押す前にタイヤが空転していのを確認してください。プロポの裏のボタンを変換しAIモードにすると動き始めます。')
title9 = ipywidgets.HTML('<b>【9.カメラのリリース】</b> 走行を終了する場合は、カメラのリリースを実行し、Notebookをshutdownしてください。')

data_collection_widget = ipywidgets.VBox([
    separator,
    title1,
    ipywidgets.HBox([model_a_widget,model_a_time_widget,load_a_button]),
    process_widget,
    separator,
    title2,
    ipywidgets.HBox([model_b_widget,model_b_time_widget,load_b_button]),
    process_widget,
    separator,
    title3,
    ipywidgets.HBox([model_p_widget,model_p_time_widget,load_p_button]),
    process_widget,
    separator,
    
    title4,
    ipywidgets.HBox([model_c_widget,model_c_time_widget,load_c_button]),
    process_widget,
    separator,
    
    title5,
    ipywidgets.HBox([steering_gain_slider]),
    separator,
    title6,
    ipywidgets.HBox([speed_dropbox]),
    ipywidgets.HBox([speed_raw_slider,speed_gain_slider]),
    ipywidgets.HBox([speed_low_gain_slider]),
    separator,
    title7,
    ipywidgets.HBox([record_box,name_widget]),
    separator,
    title8,
    ipywidgets.HBox([used_memory_widget,total_memory_widget,memory_button]),
    ipywidgets.HBox([run_button, stop_button]),
    process_widget,
    separator,
    title9,
    release_button,
    process_widget,
])
display(data_collection_widget)